In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

df = pd.read_excel('kerevit_median_final.xlsx')

X = df[['W', 'KB', 'KE', 'AB', 'AE', 'K_Sag', 'K_Sol', 'U_Sag', 'U_Sol', 'KE_Sag', 'KE_Sol']]
y = df['CINSIYET']

pipe = Pipeline([
    ('scaler', StandardScaler()),  
    ('rf', RandomForestClassifier())  
])

param_grid = {'rf__max_depth': range(1, 11)}

grid_search = GridSearchCV(pipe, param_grid, cv=10, scoring='accuracy', verbose=1, n_jobs=-1)
grid_search.fit(X, y)

print("En iyi parametreler: ", grid_search.best_params_)
print("En iyi doğruluk: ", grid_search.best_score_)

C:\Users\Yasin\anaconda3\lib\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
C:\Users\Yasin\anaconda3\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.2' currently installed).
  from pandas.core import (


Fitting 10 folds for each of 10 candidates, totalling 100 fits
En iyi parametreler:  {'rf__max_depth': 9}
En iyi doğruluk:  0.7037878787878789


# Eğitim ve Test Aşaması

In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
import joblib

df = pd.read_excel('kerevit_median_final.xlsx')

X = df[['W', 'KB', 'KE', 'AB', 'AE', 'K_Sag', 'K_Sol', 'U_Sag', 'U_Sol', 'KE_Sag', 'KE_Sol']]
y = df['CINSIYET']
sira = df['Sira'] 

kf = KFold(n_splits=10, shuffle=False)
conf_matrices = []
fold = 1

results = pd.DataFrame(columns=['Sira', 'Tahmin', 'Gerçek'])

best_rf = RandomForestClassifier(max_depth=9)

for train_index, test_index in kf.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    sira_test = sira.iloc[test_index]
    
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    
    best_rf.fit(X_train, y_train)
    
    model_filename = f'rf_model_fold_{fold}.joblib'
    joblib.dump(best_rf, model_filename)
    
    y_pred = best_rf.predict(X_test)
    
    cm = confusion_matrix(y_test, y_pred)
    conf_matrices.append(cm)
    
    print(f"Fold {fold}:")
    print("Accuracy:", accuracy_score(y_test, y_pred))
    print("Classification Report:\n", classification_report(y_test, y_pred))
    print("Confusion Matrix:\n", cm)
    print("\n")
    
    fold_results = pd.DataFrame({
        'Sira': sira_test,
        'Tahmin': y_pred,
        'Gerçek': y_test
    })
    results = pd.concat([results, fold_results])
    
    fold += 1

total_conf_matrix = np.zeros((2, 2), dtype=int) 
for i, cm in enumerate(conf_matrices, 1):
    print(f"Confusion Matrix for Fold {i}:\n{cm}\n")
    total_conf_matrix += cm

print("Total Confusion Matrix:\n", total_conf_matrix)

TN, FP, FN, TP = total_conf_matrix.ravel()

accuracy = (TP + TN) / (TP + TN + FP + FN)
precision = TP / (TP + FP) if (TP + FP) != 0 else 0
recall = TP / (TP + FN) if (TP + FN) != 0 else 0
specificity = TN / (TN + FP) if (TN + FP) != 0 else 0
f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) != 0 else 0

print(f"Overall Accuracy: {accuracy:.4f}")
print(f"Overall Precision: {precision:.4f}")
print(f"Overall Recall: {recall:.4f}")
print(f"Overall Specificity: {specificity:.4f}")
print(f"Overall F1 Score: {f1_score:.4f}")

results.to_excel('rf_predict_median.xlsx', index=False)

Fold 1:
Accuracy: 0.75
Classification Report:
               precision    recall  f1-score   support

           D       0.75      0.86      0.80         7
           E       0.75      0.60      0.67         5

    accuracy                           0.75        12
   macro avg       0.75      0.73      0.73        12
weighted avg       0.75      0.75      0.74        12

Confusion Matrix:
 [[6 1]
 [2 3]]


Fold 2:
Accuracy: 0.6666666666666666
Classification Report:
               precision    recall  f1-score   support

           D       0.67      0.86      0.75         7
           E       0.67      0.40      0.50         5

    accuracy                           0.67        12
   macro avg       0.67      0.63      0.62        12
weighted avg       0.67      0.67      0.65        12

Confusion Matrix:
 [[6 1]
 [3 2]]


Fold 3:
Accuracy: 0.6363636363636364
Classification Report:
               precision    recall  f1-score   support

           D       0.62      0.83      0.71       

C:\Users\Yasin\AppData\Local\Temp/ipykernel_4992/1772251310.py:78: UserWarning: Pandas requires version '3.0.5' or newer of 'xlsxwriter' (version '3.0.1' currently installed).
  results.to_excel('rf_predict_median.xlsx', index=False)
